In [0]:
%tensorflow_version 2.x

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as py

In [0]:
from  tensorflow.keras.preprocessing import sequence
from  tensorflow.keras.models import Sequential
from  tensorflow.keras.layers import Dense,Embedding, Dropout, Activation
from  tensorflow.keras.layers import LSTM,GRU
from  tensorflow.keras.layers import Conv1D, MaxPooling1D
from  tensorflow.keras.datasets import imdb

In [4]:
(x_train,y_train) ,(x_test,y_test) = imdb.load_data(num_words = 10000)

17465344/17464789 [==============================] - 0s 0us/step


In [0]:
##看頻率最高的450個字
max_words = 450


x_train = sequence.pad_sequences(x_train,maxlen = max_words)
x_test  = sequence.pad_sequences(x_test,maxlen=max_words)

In [0]:
##model 1 


##修改LSTM變成GRU
##修改dropout、修改recurrent_dropout
##修改EPOCH
##修改BATCH_SIZE =128

model = Sequential()
model.add(Embedding(10000,128))
model.add(GRU(256, dropout=0.4, recurrent_dropout=0.25))
model.add(Dense(500, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
gru (GRU)                    (None, 256)               296448    
_________________________________________________________________
dense (Dense)                (None, 500)               128500    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 501       
Total params: 1,705,449
Trainable params: 1,705,449
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=5, batch_size=128)

Epoch 1/5
196/196 [==============================] - 1331s 7s/step - loss: 0.5232 - accuracy: 0.7246 - val_loss: 0.3401 - val_accuracy: 0.8587
Epoch 2/5
196/196 [==============================] - 1329s 7s/step - loss: 0.2846 - accuracy: 0.8850 - val_loss: 0.4894 - val_accuracy: 0.8412
Epoch 3/5
196/196 [==============================] - 1304s 7s/step - loss: 0.2327 - accuracy: 0.9087 - val_loss: 0.3180 - val_accuracy: 0.8670
Epoch 4/5
196/196 [==============================] - 1291s 7s/step - loss: 0.1473 - accuracy: 0.9456 - val_loss: 0.3354 - val_accuracy: 0.8666
Epoch 5/5
196/196 [==============================] - 1303s 7s/step - loss: 0.1061 - accuracy: 0.9607 - val_loss: 0.3853 - val_accuracy: 0.8795


In [11]:
##model 2 
##CNN + LSTM 

#######    First layer    ####### 
####  Embedding(can only be the first layer) 
#Meaning : turn  positive integer into vector of fixed size 
#Parameter : 
#1.input_dim = 大於等於0的整數，dic長度
#2.output_dim = 大於等於0的整數，dense Embedding維度
#3.input_length:當輸入的序列長度固定，此數值則為該序列長度，Flatten層前為此層再來才為Dense層


#######    Second layer    ####### 
####  Dropout (目的是避免神經網路記答案(overfitting)) 
#Parameter:
#1.rate = 在0-1之間的數值，表示需要丟棄的input比例
#2.seed = 隨機整數(作用與r的set seed相同)

#######    Third layer    ####### 
####  Conv1D(ex time series convolutional layer)
#Parameter:
#1.filter = integer ,dimension of  output space
#2.kernal_size = integer , specifying the length of the 1D convolution window
#3.strides = integer ,length of the convolution
#4.padding='valid'=>輸出矩陣大小不用一樣

#######    Fourth layer    ####### 
#### MaxPooling1D
#Meaning :Max pooling operation for temporal data
#Parameter:
#1.pooling size= integer ,size of the max pooling windows.




max_input_dim = 10000
maxlen = 200
embedding_size = 128
kernel_size = 50
filters = 32
pool_size = 3

model2 = Sequential()
model2.add(Embedding(max_input_dim, embedding_size, input_length=maxlen))
model2.add(Dropout(0.3))
model2.add(Conv1D(filters,kernel_size,padding='valid',activation='relu',strides=1))
model2.add(MaxPooling1D(pool_size=pool_size))
model2.add(LSTM(128))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()



##計算每層

# embedding_2 input layer  (1) input: (None,200) (2)output:(None,200)  
# embedding_2 embedding  (1) input: (None,200) (2)output:(None,200,128) {Param  = 10000維*128  因為標準embeding 只做線性的(=不做bias)}
# dropout  (1) input: (None,200,128) (2)output:(None,200,128)
# Conv1D  (1) input: (None,200,128) (2)output:(None,151,32)  {Param = (50*128+1)*32}
# max_pooling1D  (1) input: (None,151,32) (2)output:(None, 50, 32)   
# LSTM  (1) input: (None,50,32) (2)output:(None,128) {(128+32+1)*4*128}
# Dense  (1) input: (None,128) (2)output:(None,1)    {128+1(bias)=129}



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 128)          1280000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 151, 32)           204832    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,567,393
Trainable params: 1,567,393
Non-trainable params: 0
____________________________________________

In [12]:
model2.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=2, batch_size=256)

Epoch 1/2
98/98 [==============================] - 589s 6s/step - loss: 0.4322 - accuracy: 0.7875 - val_loss: 0.3250 - val_accuracy: 0.8599
Epoch 2/2
98/98 [==============================] - 592s 6s/step - loss: 0.2280 - accuracy: 0.9126 - val_loss: 0.3169 - val_accuracy: 0.8698
